# Dataset Creator

## How to get data

1. Data taken from http://www.naturalstattrick.com/

2. Go in players -> individiual

3. Use these settings

<img src="../figures/data-ex.png"
     alt="alt"
     style="float: left; margin: 20px;" />
     

4. Click on `CSV (all)` and save under the name `nhl-19-20-ind-rates.csv`

5. Change from `All Strengths` to `Power Play`, click on `CSV (all)` and save under the name `nhl-19-20-ind-pp-rates.csv`

6. Change from `Individual` to `On Ice`, click on `CSV (all)` and save under the name `nhl-19-20-ice-pp-rates.csv`

7. Change from `Power Play` to `All Strengths`, click on `CSV (all)` and save under the name `nhl-19-20-ice-rates.csv`

8. Go in players -> bios

9. Use these settings

<img src="../figures/data-bio.png"
     alt="alt"
     style="float: left; margin: 20px;" />
     
10. Click on `CSV (all)` and save under the name `nhl-19-20-bio.csv`

11. Put the data in `../data/nhl-stats_rates`

## Note

* CTRL-F and search for `TODO` to modify parts dependant of the nhl season.

* Yearly projections are made by a redditor `u/ChuckLemieux`, watch out for his google spreadsheet. It aggregates a bunch of projections.

## 1. Preparation
---

In [1]:
import pandas as pd
import numpy as np
import re
import math
import pickle
import glob
import xgboost
import tsfresh
import os
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from pprint import pprint
from sklearn import svm
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import export_graphviz
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score, r2_score, log_loss
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

sns.set_context('notebook')
sns.set_color_codes()
plot_kwds = {'alpha' : 0.5, 's' : 100, 'linewidths':0}
large = 22; med = 16; small = 12
params = {'axes.titlesize': large,
          'legend.fontsize': med,
          'figure.figsize': (16, 10),
          'axes.labelsize': med,
          'axes.titlesize': med,
          'xtick.labelsize': med,
          'ytick.labelsize': med,
          'figure.titlesize': large}
plt.rcParams.update(params)
plt.style.use('seaborn-whitegrid')
sns.set_style("white")

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# Paths
data_folder = '../data/'

## 2. Read datasets
---

In [3]:
def read_dataset(file_paths_regex):
    data_paths = glob.glob(file_paths_regex)
    data_paths.sort()
    
    data = pd.DataFrame()
    for file_path in data_paths:
        file = pd.read_csv(file_path, sep=',', encoding="utf-8-sig")
        file['Year'] = '20' + file_path.split('-')[2]
        file = file.rename(columns={'Unnamed: 0': 'Index'}).set_index('Index')
        data = pd.concat([data, file], sort=False)
    data['Name'] = data.apply(lambda player: re.sub(r'\W+', '', f'{player["Player"]})').upper(), axis=1)
    data.set_index('Name', inplace=True)
    return data

In [4]:
# Read the 3 kinds of datasets
individuals = read_dataset(os.path.join(data_folder, 'nhl-stats_rates/*ind-rates.csv'))
on_ice = read_dataset(os.path.join(data_folder, './nhl-stats_rates/*ice-rates.csv'))
on_ice.drop(['GP', 'TOI'], axis=1, inplace=True) # Because already in individuals
on_ice_pp = read_dataset(os.path.join(data_folder, './nhl-stats_rates/*ind-pp.csv'))
on_ice_pp.drop(['GP'], axis=1, inplace=True) # Because already in individuals
new_cols = ['_'.join([col, 'PP']) for col in on_ice_pp.columns[3:]]
new_cols = ['Player', 'Team', 'Position'] + new_cols
new_cols[-1] = 'Year'
on_ice_pp.columns = new_cols
biography = read_dataset(os.path.join(data_folder, './nhl-stats_rates/*bio.csv'))

# Merge the 3 datasets together
tmp = pd.merge(biography, individuals, on=['Player', 'Team', 'Position', 'Year'])
tmp = pd.merge(tmp, on_ice, on=['Player', 'Team', 'Position', 'Year'])
nhl_players = pd.merge(tmp, on_ice_pp, on=['Player', 'Team', 'Position', 'Year'])

# List of columns
cols = list(nhl_players)
                                    
# Move the column to head of list using index, pop and insert Year first
cols.insert(0, cols.pop(cols.index('Year')))

# Reorder
nhl_players = nhl_players.loc[:, cols]
nhl_players = nhl_players.sort_values(['Player', 'Year'])

nhl_players.head(10)

,Year,Player,Team,Position,Age,Date of Birth,Birth City,Birth State/Province,Birth Country,Nationality,Height (in),Weight (lbs),Draft Year,Draft Team,Draft Round,Round Pick,Overall Draft Position,GP,TOI,TOI/GP_x,Goals/60,Total Assists/60,First Assists/60,Second Assists/60,Total Points/60,IPP,Shots/60,SH%,iCF/60,iFF/60,iSCF/60,iHDCF/60,Rush Attempts/60,Rebounds Created/60,PIM/60,Total Penalties/60,Minor/60,Major/60,Misconduct/60,Penalties Drawn/60,Giveaways/60,Takeaways/60,Hits/60,Hits Taken/60,Shots Blocked/60,Faceoffs Won/60,Faceoffs Lost/60,Faceoffs %,ixG/60,TOI/GP_y,CF/60,CA/60,CF%,FF/60,FA/60,FF%,SF/60,SA/60,SF%,GF/60,GA/60,GF%,SCF/60,SCA/60,SCF%,HDCF/60,HDCA/60,HDCF%,HDGF/60,HDGA/60,HDGF%,MDCF/60,MDCA/60,MDCF%,MDGF/60,MDGA/60,MDGF%,LDCF/60,LDCA/60,LDCF%,LDGF/60,LDGA/60,LDGF%,On-Ice SH%,On-Ice SV%,PDO,Off. Zone Starts/60,Neu. Zone Starts/60,Def. Zone Starts/60,On The Fly Starts/60,Off. Zone Start %,Off. Zone Faceoffs/60,Neu. Zone Faceoffs/60,Def. Zone Faceoffs/60,Off. Zone Faceoff %,xGF/60,xGA/60,xGF%,TOI_PP,Goals_PP,Total Assists_PP,First Assists_PP,Second Assists_PP,Total Points_PP,IPP_PP,Shots_PP,SH%_PP,ixG_PP,iCF_PP,iFF_PP,iSCF_PP,iHDCF_PP,Rush Attempts_PP,Rebounds Created_PP,PIM_PP,Total Penalties_PP,Minor_PP,Major_PP,Misconduct_PP,Penalties Drawn_PP,Giveaways_PP,Takeaways_PP,Hits_PP,Hits Taken_PP,Shots Blocked_PP,Faceoffs Won_PP,Faceoffs Lost_PP,Faceoffs %_PP
1630,2016,A.J. Greer,COL,L,21,1996-12-14,Joliette,QC,CAN,CAN,75,210,2015,COL,2,9,39,5,68.283333,13.656667,0.00,0.88,0.00,0.88,0.88,100.00,7.03,0.00,9.67,8.79,6.15,3.51,0.00,1.76,3.51,1.76,1.76,0.00,0.00,1.76,1.76,0.88,3.51,5.27,2.64,0.00,0.00,-,NaN,13.656667,57.99,50.09,53.66,44.81,33.39,57.3,31.63,28.12,52.94,0.88,2.64,25.00,27.24,12.30,68.89,13.18,3.51,78.95,0.88,0.00,100.00,18.75,11.72,61.54,0.00,2.34,0.00,36.81,39.35,48.33,0.00,1.27,0.00,2.78,90.63,0.934,11.42,20.21,12.30,45.69,48.15,28.12,27.24,25.48,52.46,NaN,NaN,NaN,2.866667,0,0,0,0,0,-,2,0.00,0.42,2,2,2,1,0,1,2,1,1,0,0,0,0,0,0,0,0,0,0,-
2441,2017,A.J. Greer,COL,L,21,1996-12-14,Joliette,QC,CAN,CAN,75,210,2015,COL,2,9,39,17,125.683333,7.393137,0.00,1.43,0.95,0.48,1.43,75.00,6.21,0.00,9.55,7.16,3.82,1.43,0.48,0.00,13.84,4.30,3.34,0.48,0.48,3.34,0.95,1.43,14.32,7.64,2.39,0.48,0.95,15.91,NaN,7.393137,42.01,61.58,40.55,32.46,47.74,40.48,25.78,34.85,42.52,1.91,0.95,66.67,21.96,25.78,46,8.12,12.41,39.53,0.95,0.48,66.67,18.46,17.82,50.88,0.64,0.00,100.00,24.13,41.37,36.84,0.69,0.69,50.00,7.41,97.26,1.047,11.46,10.03,10.03,58.24,53.33,17.19,16.23,15.75,52.17,NaN,NaN,NaN,2.783333,0,0,0,0,0,-,0,-,0.00,0,0,0,0,0,0,7,2,1,1,0,2,0,0,1,0,0,0,0,-
0,2014,Aaron Ekblad,FLA,D,22,1996-02-07,Windsor,ON,CAN,CAN,76,220,2014,FLA,1,1,1,81,1766.616667,21.810082,0.41,0.92,0.34,0.58,1.32,44.32,5.77,7.06,11.92,8.08,2.65,0.48,0.17,0.95,1.09,0.54,0.54,0.00,0.00,0.51,1.39,0.88,3.70,4.62,2.72,0.00,0.00,-,NaN,21.810082,59.84,44.02,57.62,46.16,34.57,57.17,35.32,25.17,58.39,2.99,1.97,60.27,26.83,19.05,58.48,10.05,8.02,55.64,1.46,0.98,59.72,22.37,14.72,60.32,1.13,0.72,60.98,41.94,30.81,57.65,0.98,0.54,64.52,8.46,92.17,1.006,11.65,10.32,5.26,38.55,68.88,26.76,17.29,13.69,66.16,NaN,NaN,NaN,226.550000,6,7,1,6,13,59.09,39,15.38,3.49,83,58,19,5,0,7,4,2,2,0,0,0,2,3,6,6,2,0,0,-
803,2015,Aaron Ekblad,FLA,D,22,1996-02-07,Windsor,ON,CAN,CAN,76,220,2014,FLA,1,1,1,78,1690.816667,21.677137,0.53,0.75,0.28,0.46,1.28,38.71,6.46,8.24,11.50,8.52,3.05,0.82,0.18,0.57,1.45,0.67,0.64,0.04,0.00,0.32,1.45,0.85,3.09,4.47,2.09,0.00,0.00,-,NaN,21.677137,54.01,43.61,55.33,42.16,34.53,54.97,31.23,24.49,56.05,3.30,2.13,60.78,25.76,19.73,56.63,10.57,8.41,55.70,1.70,1.21,58.54,20.25,15.09,57.30,1.32,0.80,62.22,36.80,29.22,55.75,0.72,0.31,70.00,10.57,91.30,1.019,11.75,13.02,7.77,32.93,60.18,24.20,21.01,15.79,60.51,NaN,NaN,NaN,221.533333,3,6,2,4,9,39.13,38,7.89,3.42,79,53,20,5,0,5,4,2,2,0,0,0,3,1,5,9,1,0,0,-
1631,2016,Aaron Ekblad,FLA,D,22,1996-02-07,Windsor,ON,CAN,CAN,76,220,2014,FLA,1,1,1,68,1459.283333,21.460049,0.41,0.45,0.16,0.29,0.86,38.89,9.25,4.44,16.78,12.17,4.56,0.66

## 3. Preprocessing
---

In [5]:
def get_draft_year(player):
    if player['Draft Year'] == '-':
        return int(player['Date of Birth'].split('-')[0]) + 18
    else:
        return player['Draft Year']
    
def get_draft_position(player):
    if player['Overall Draft Position'] == '-':
        return 0
    else:
        return player['Overall Draft Position']   

### 3.1 Clean Data

In [6]:
# Fix column names
# There is some weird characters for some column names
cols = nhl_players.columns
cols = cols.map(lambda x: x.replace(u'\xa0', u' ') if isinstance(x, (str)) else x)
nhl_players.columns = cols

In [7]:
# Fill missing data
# Age infered based on birth year and current year of season played
nhl_players['Date of Birth'] = nhl_players['Date of Birth'].replace({'-': '2000-01-01'})
nhl_players['Age'] = nhl_players.apply(lambda plyr: int(plyr['Year'])-int(plyr['Date of Birth'].split('-')[0]), axis=1)

# If not draft year, infered based on date of birth + 18 years
nhl_players['Draft Year'] = nhl_players.apply(lambda plyr: get_draft_year(plyr), axis=1)

# If no overall draft position, we return 0
nhl_players['Overall Draft Position'] = nhl_players.apply(lambda plyr: get_draft_position(plyr), axis=1)

# Real total points
nhl_players['Total Points'] = (nhl_players['Total Points/60'] * nhl_players['TOI'] / 60).astype(int)
nhl_players['Total Points/GP'] = nhl_players['Total Points']/nhl_players['GP']

# If no value is given for a particular stat, we insert the mean of that stat
for col in ['GP', 'TOI', 'Goals/60', 'Total Assists/60', 'First Assists/60', 'Second Assists/60', 'Total Points/60', 
            'IPP', 'Shots/60', 'SH%', 'iCF/60', 'iFF/60', 'iSCF/60', 'iHDCF/60', 'Rush Attempts/60', 'Rebounds Created/60', 
            'PIM/60', 'Total Penalties/60', 'Minor/60', 'Major/60', 'Misconduct/60', 'Penalties Drawn/60', 'Giveaways/60', 
            'Takeaways/60', 'Hits/60', 'Hits Taken/60', 'Shots Blocked/60', 'Faceoffs Won/60', 'Faceoffs Lost/60', 
            'Faceoffs %', 'CF/60', 'CA/60', 'CF%', 'FF/60', 'FA/60', 'FF%', 'SF/60', 'SA/60', 'SF%', 'GF/60', 'GA/60', 'GF%', 
            'SCF/60', 'SCA/60', 'SCF%', 'HDCF/60', 'HDCA/60', 'HDCF%', 'HDGF/60', 'HDGA/60', 'HDGF%', 'MDCF/60', 'MDCA/60', 
            'MDCF%', 'MDGF/60', 'MDGA/60', 'MDGF%', 'LDCF/60', 'LDCA/60', 'LDCF%', 'LDGF/60', 'LDGA/60', 'LDGF%', 
            'On-Ice SH%', 'On-Ice SV%', 'PDO', 'Off. Zone Starts/60',
            'Neu. Zone Starts/60', 'Def. Zone Starts/60', 'On The Fly Starts/60',
            'Off. Zone Start %', 'Off. Zone Faceoffs/60', 'Neu. Zone Faceoffs/60',
            'Def. Zone Faceoffs/60', 'Off. Zone Faceoff %']:
    mean = nhl_players[~nhl_players[col].isin(['-'])][col].astype(float).mean()
    nhl_players[col].replace('-', 0, inplace=True)
    
for col in ['TOI_PP', 'Goals_PP', 'Total Assists_PP', 'First Assists_PP', 'Second Assists_PP', 'Total Points_PP', 
            'IPP_PP', 'Shots_PP', 'SH%_PP', 'iCF_PP', 'iFF_PP', 'iSCF_PP', 'iHDCF_PP', 'Rush Attempts_PP', 
            'Rebounds Created_PP', 'PIM_PP', 'Total Penalties_PP', 'Minor_PP', 'Major_PP', 'Misconduct_PP', 'Penalties Drawn_PP', 
            'Giveaways_PP', 'Takeaways_PP', 'Hits_PP', 'Hits Taken_PP', 'Shots Blocked_PP', 'Faceoffs Won_PP', 'Faceoffs Lost_PP', 
            'Faceoffs %_PP']:
    mean = nhl_players[~nhl_players[col].isin(['-'])][col].astype(float).mean()
    nhl_players[col].replace('-', 0, inplace=True)
    
# To numeric
nhl_players[nhl_players.columns[16:]] = nhl_players[nhl_players.columns[16:]].apply(pd.to_numeric)
nhl_players.Year = nhl_players.Year.apply(pd.to_numeric)

### 3.2 Split target values

In [10]:
# Rework to get the points from the last season played of all player, and divide by games played, this is the target value
# And remove the features from this last season (so the lines of this player for his last season)

# Get target value (pts/gp in 2017)
# Sort and get pts/gp
nhl_players.sort_values(['Player', 'Year'], inplace=True)
nhl_players['PTS_expected'] = round(82*(nhl_players['Total Points']/nhl_players['GP']))

# Select the last season played by each player
total_points_df = nhl_players.groupby('Player').tail(1)
total_points_df_sub = total_points_df[['Player', 'PTS_expected']].set_index('Player')
total_points = total_points_df_sub['PTS_expected']

# Remove target value instances
# nhl_players = pd.merge(nhl_players, total_points_df, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)

# Keep only the last 5 seasons of each players, to take into account their current pace and not their entire career pace
# eg: Chara is doing less points at 39 than 25
nhl_players = nhl_players.groupby('Player').tail(5)

### 3.3 Remove unecessary features

In [11]:
# Split features
# Keep in memory the player and year order
nhl_players_features_player = nhl_players['Player']
nhl_players_features_year = nhl_players['Year']

# Drop unecessary columns
nhl_players_features = nhl_players.drop(['Team', 'Position', 'Draft Year', 'Date of Birth', 
                                         'Birth City', 'Birth State/Province', 'Birth Country', 
                                         'Nationality', 'Height (in)', 'Weight (lbs)', 
                                         'Draft Team', 'Draft Round', 'Round Pick', 'IPP', 'Overall Draft Position',
                                         'PIM/60', 'Total Penalties/60', 'Minor/60', 'Major/60', 'Misconduct/60',
                                         'Hits/60', 'Hits Taken/60', 'Shots Blocked/60',
                                         'Goals/60', 'Total Assists/60', 'First Assists/60', 'Second Assists/60',
                                         'Goals_PP', 'Total Assists_PP', 'First Assists_PP', 'Second Assists_PP',
                                         'Rush Attempts/60', 'Rebounds Created/60', 'Penalties Drawn/60', 'Giveaways/60',
                                         'Rush Attempts_PP', 'Rebounds Created_PP', 'PIM_PP', 'Total Penalties_PP', 'Minor_PP', 'Major_PP', 'Misconduct_PP', 'Penalties Drawn_PP',
                                         'Takeaways/60', 'Faceoffs Won/60', 'Faceoffs Lost/60', 'Faceoffs %', 'TOI/GP_y',
                                         'Neu. Zone Starts/60', 'Def. Zone Starts/60', 
                                         'On The Fly Starts/60', 'Neu. Zone Faceoffs/60', 'Def. Zone Faceoffs/60',
                                         'Giveaways_PP', 'Takeaways_PP', 'Hits_PP', 'Hits Taken_PP', 'Shots Blocked_PP', 
                                         'Faceoffs Won_PP', 'Faceoffs Lost_PP', 'Faceoffs %_PP',
                                         'CA/60', 'FA/60', 'SA/60', 'GA/60', 'SCA/60', 'HDCA/60', 'HDGA/60', 'MDCA/60', 'MDGA/60', 'LDCA/60', 'LDGA/60', 'SH%', 'On-Ice SV%',
                                         'HDCF/60', 'HDGF/60', 'MDCF/60', 'MDGF/60', 'LDCF/60', 'LDGF/60',
                                         'HDCF%', 'HDGF%', 'MDCF%', 'MDGF%', 'LDCF%', 'LDGF%'], axis=1)

# Sort Values
nhl_players_features = nhl_players_features.fillna(0)
nhl_players_features.sort_values(['Player', 'Year'], inplace=True)

### 3.4 Feature engineering

In [12]:
def normalizeFeature(feature):
    #Normalized Data
    return (feature-min(feature))/(max(feature)-min(feature))

def getPlayerScore(player):
    name = player.name
    return normalizeFeature(features_filtered_direct['CF%__mean'])[name] + normalizeFeature(features_filtered_direct['FF%__mean'])[name] + \
            normalizeFeature(features_filtered_direct['GF%__mean'])[name] + normalizeFeature(features_filtered_direct['GP__mean'])[name] + \
            normalizeFeature(features_filtered_direct['Off. Zone Faceoff %__mean'])[name] + normalizeFeature(features_filtered_direct['Off. Zone Start %__mean'])[name] + \
            normalizeFeature(features_filtered_direct['On-Ice SH%__mean'])[name] - normalizeFeature(features_filtered_direct['PDO__mean'])[name] + \
            normalizeFeature(features_filtered_direct['PTS_expected__mean'])[name] + normalizeFeature(features_filtered_direct['SCF%__mean'])[name] + \
            normalizeFeature(features_filtered_direct['SF%__mean'])[name] + \
            normalizeFeature(features_filtered_direct['Shots/60__mean'])[name] + normalizeFeature(features_filtered_direct['TOI__mean'])[name] + \
            normalizeFeature(features_filtered_direct['TOI_PP__mean'])[name] + normalizeFeature(features_filtered_direct['iSCF/60__mean'])[name]

In [13]:
# Feature Extractor
print('Extract features...')
from tsfresh.feature_extraction import *
features_filtered_direct = tsfresh.extract_features(nhl_players_features, 
                                                    column_id='Player', 
                                                    column_sort='Year',
                                                    default_fc_parameters=tsfresh.feature_extraction.settings.MinimalFCParameters())
 
# Convert NaN and inf
print('Impute NaN and Inf, remove zeros columns...')
features_filtered_direct = tsfresh.utilities.dataframe_functions.impute(features_filtered_direct)

# Remove zeros columns
features_filtered_direct = features_filtered_direct.loc[:, (features_filtered_direct != 0).any(axis=0)]

# Remove columns with same values
cols = list(features_filtered_direct)
nunique = features_filtered_direct.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
features_filtered_direct = features_filtered_direct.drop(cols_to_drop, axis=1)
print('Done.')

# Select relevant features
features_filtered_direct = features_filtered_direct.drop(list(filter(re.compile(".*_length").match, features_filtered_direct.columns)), axis=1)
features_filtered_direct = features_filtered_direct.drop(list(filter(re.compile(".*_sum_values").match, features_filtered_direct.columns)), axis=1)
features_filtered_direct = features_filtered_direct.drop(list(filter(re.compile(".*_standard_deviation").match, features_filtered_direct.columns)), axis=1)
features_filtered_direct = features_filtered_direct.drop(list(filter(re.compile("Age*").match, features_filtered_direct.columns)), axis=1)
# tsfresh.select_features(features_filtered_direct, total_points[:8], ml_task='regression')

Extract features...


Feature Extraction: 100%|██████████| 20/20 [00:05<00:00,  4.46it/s]


Impute NaN and Inf, remove zeros columns...
Done.


In [14]:
# Add current age, draft position
features_filtered_direct['Age'] = nhl_players.groupby('Player').tail(1).Age.values
features_filtered_direct['Overall Draft Position'] = nhl_players.groupby('Player').tail(1)['Overall Draft Position'].values

# Add stats from last 5 years
year_by_year = nhl_players_features.groupby('Player').tail(5).pivot_table(index='Player', columns='Year', values=nhl_players_features.columns)
year_by_year.columns = ['_'.join(str(i) for i in col) for col in year_by_year.columns.values]

# TODO: make sur we keep only 5 years
for yr in ['2007', '2008', '2009', '2010', '2011', '2012', '2013']:
    year_by_year = year_by_year.drop(list(filter(re.compile(f".*_{yr}").match, year_by_year.columns)), axis=1)
features_filtered_direct = pd.concat([features_filtered_direct, year_by_year], axis=1)
features_filtered_direct['Position'] = nhl_players.groupby('Player').tail(1).Position.values
features_filtered_direct['Team'] = nhl_players.groupby('Player').tail(1).Team.values

In [15]:
# Get player score
features_filtered_direct['Score'] = features_filtered_direct.apply(lambda player: getPlayerScore(player), axis=1)

# Saving feature names for later use
feature_list = list(features_filtered_direct.columns)
features_filtered_direct.sort_values('Score', ascending=False).head(10)

,CF%__maximum,CF%__mean,CF%__median,CF%__minimum,CF%__variance,CF/60__maximum,CF/60__mean,CF/60__median,CF/60__minimum,CF/60__variance,FF%__maximum,FF%__mean,FF%__median,FF%__minimum,FF%__variance,FF/60__maximum,FF/60__mean,FF/60__median,FF/60__minimum,FF/60__variance,GF%__maximum,GF%__mean,GF%__median,GF%__minimum,GF%__variance,GF/60__maximum,GF/60__mean,GF/60__median,GF/60__minimum,GF/60__variance,GP__maximum,GP__mean,GP__median,GP__minimum,GP__variance,IPP_PP__maximum,IPP_PP__mean,IPP_PP__median,IPP_PP__minimum,IPP_PP__variance,Off. Zone Faceoff %__maximum,Off. Zone Faceoff %__mean,Off. Zone Faceoff %__median,Off. Zone Faceoff %__minimum,Off. Zone Faceoff %__variance,Off. Zone Faceoffs/60__maximum,Off. Zone Faceoffs/60__mean,Off. Zone Faceoffs/60__median,Off. Zone Faceoffs/60__minimum,Off. Zone Faceoffs/60__variance,Off. Zone Start %__maximum,Off. Zone Start %__mean,Off. Zone Start %__median,Off. Zone Start %__minimum,Off. Zone Start %__variance,Off. Zone Starts/60__maximum,Off. Zone Starts/60__mean,Off. Zone Starts/60__median,Off. Zone Starts/60__minimum,Off. Zone Starts/60__variance,On-Ice SH%__maximum,On-Ice SH%__mean,On-Ice SH%__median,On-Ice SH%__minimum,On-Ice SH%__variance,PDO__maximum,PDO__mean,PDO__median,PDO__minimum,PDO__variance,PTS_expected__maximum,PTS_expected__mean,PTS_expected__median,PTS_expected__minimum,PTS_expected__variance,SCF%__maximum,SCF%__mean,SCF%__median,SCF%__minimum,SCF%__variance,SCF/60__maximum,SCF/60__mean,SCF/60__median,SCF/60__minimum,SCF/60__variance,SF%__maximum,SF%__mean,SF%__median,SF%__minimum,SF%__variance,SF/60__maximum,SF/60__mean,SF/60__median,SF/60__minimum,SF/60__variance,SH%_PP__maximum,SH%_PP__mean,SH%_PP__median,SH%_PP__minimum,SH%_PP__variance,Shots/60__maximum,Shots/60__mean,Shots/60__median,Shots/60__minimum,Shots/60__variance,Shots_PP__maximum,Shots_PP__mean,Shots_PP__median,Shots_PP__minimum,Shots_PP__variance,TOI/GP_x__maximum,TOI/GP_x__mean,TOI/GP_x__median,TOI/GP_x__minimum,TOI/GP_x__variance,TOI_PP__maximum,TOI_PP__mean,TOI_PP__median,TOI_PP__minimum,TOI_PP__variance,TOI__maximum,TOI__mean,TOI__median,TOI__minimum,TOI__variance,Total Points/60__maximum,Total Points/60__mean,Total Points/60__median,Total Points/60__minimum,Total Points/60__variance,Total Points/GP__maximum,Total Points/GP__mean,Total Points/GP__median,Total Points/GP__minimum,Total Points/GP__variance,Total Points_PP__maximum,Total Points_PP__mean,Total Points_PP__median,Total Points_PP__minimum,Total Points_PP__variance,Total Points__maximum,Total Points__mean,Total Points__median,Total Points__minimum,Total Points__variance,iCF/60__maximum,iCF/60__mean,iCF/60__median,iCF/60__minimum,iCF/60__variance,iCF_PP__maximum,iCF_PP__mean,iCF_PP__median,iCF_PP__minimum,iCF_PP__variance,iFF/60__maximum,iFF/60__mean,iFF/60__median,iFF/60__minimum,iFF/60__variance,iFF_PP__maximum,iFF_PP__mean,iFF_PP__median,iFF_PP__minimum,iFF_PP__variance,iHDCF/60__maximum,iHDCF/60__mean,iHDCF/60__median,iHDCF/60__minimum,iHDCF/60__variance,iHDCF_PP__maximum,iHDCF_PP__mean,iHDCF_PP__median,iHDCF_PP__minimum,iHDCF_PP__variance,iSCF/60__maximum,iSCF/60__mean,iSCF/60__median,iSCF/60__minimum,iSCF/60__variance,iSCF_PP__maximum,iSCF_PP__mean,iSCF_PP__median,iSCF_PP__minimum,iSCF_PP__variance,ixG/60__maximum,ixG/60__mean,ixG/60__median,ixG/60__minimum,ixG/60__variance,ixG_PP__maximum,ixG_PP__mean,ixG_PP__median,ixG_PP__minimum,ixG_PP__variance,xGA/60__maximum,xGA/60__mean,xGA/60__median,xGA/60__minimum,xGA/60__variance,xGF%__maximum,xGF%__mean,xGF%__median,xGF%__minimum,xGF%__variance,xGF/60__maximum,xGF/60__mean,xGF/60__median,xGF/60__minimum,xGF/60__variance,Age,Overall Draft Position,Age_2014,Age_2015,Age_2016,Age_2017,Age_2018,CF%_2014,CF%_2015,CF%_2016,CF%_2017,CF%_2018,CF/60_2014,CF/60_2015,CF/60_2016,CF/60_2017,CF/60_2018,FF%_2014,FF%_2015,FF%_2016,FF%_2017,FF%_2018,FF/60_2014,FF/60_2015,FF/60_2016,FF/60_2017,FF/60_2018,GF%_2014,GF%_2015,GF%_2016,GF%_2017,GF%_2018,GF/60_2014,GF/60_2015,GF/60_2016,GF/60_2017,GF/60_2018,GP_2014,

### 3.5 Name Standardization

**TODO: Find a better way to deal with wrong names**

In [18]:
features_filtered_direct['Name'] = features_filtered_direct.apply(lambda player: re.sub(r'\W+', '', f'{player.name}').upper(), axis=1)
nhl = features_filtered_direct.copy()
nhl = nhl.set_index('Name')
nhl = nhl.rename({'BURMISTROVALEX': 'BURMISTROVALEXANDER',
                  'BURROWSALEX': 'BURROWSALEXANDRE',
                  'CAMMALLERIMIKE': 'CAMMALLERIMICHAEL',
                  'DEANGELOTONY': 'DEANGELOANTHONY',
                  'DIDOMENICOCHRIS': 'DIDOMENICOCHRISTOPHER',
                  'EDLERALEX': 'EDLERALEXANDER',
                  'ENSTROMTOBY': 'ENSTROMTOBIAS',
                  'FERLANDMICHEAL': 'FERLANDMICHAEL',
                  'HINOSTROZAVINNIE': 'HINOSTROZAVINCENT',
                  'KULEMINNIKOLAY': 'KULEMINNIKOLAI',
                  'MARCHESSAULTJONATHAN': 'AUDYMARCHESSAULTJONATHAN',
                  'MATHESONMIKE': 'MATHESONMICHAEL',
                  'MORRISSEYJOSH': 'MORRISSEYJOSHUA',
                  'PETROVICALEX': 'PETROVICALEXANDER',
                  'RADULOVALEX': 'RADULOVALEXANDER',
                  'WENNBERGALEX': 'WENNBERGALEXANDER'})
nhl.head(10)

,CF%__maximum,CF%__mean,CF%__median,CF%__minimum,CF%__variance,CF/60__maximum,CF/60__mean,CF/60__median,CF/60__minimum,CF/60__variance,FF%__maximum,FF%__mean,FF%__median,FF%__minimum,FF%__variance,FF/60__maximum,FF/60__mean,FF/60__median,FF/60__minimum,FF/60__variance,GF%__maximum,GF%__mean,GF%__median,GF%__minimum,GF%__variance,GF/60__maximum,GF/60__mean,GF/60__median,GF/60__minimum,GF/60__variance,GP__maximum,GP__mean,GP__median,GP__minimum,GP__variance,IPP_PP__maximum,IPP_PP__mean,IPP_PP__median,IPP_PP__minimum,IPP_PP__variance,Off. Zone Faceoff %__maximum,Off. Zone Faceoff %__mean,Off. Zone Faceoff %__median,Off. Zone Faceoff %__minimum,Off. Zone Faceoff %__variance,Off. Zone Faceoffs/60__maximum,Off. Zone Faceoffs/60__mean,Off. Zone Faceoffs/60__median,Off. Zone Faceoffs/60__minimum,Off. Zone Faceoffs/60__variance,Off. Zone Start %__maximum,Off. Zone Start %__mean,Off. Zone Start %__median,Off. Zone Start %__minimum,Off. Zone Start %__variance,Off. Zone Starts/60__maximum,Off. Zone Starts/60__mean,Off. Zone Starts/60__median,Off. Zone Starts/60__minimum,Off. Zone Starts/60__variance,On-Ice SH%__maximum,On-Ice SH%__mean,On-Ice SH%__median,On-Ice SH%__minimum,On-Ice SH%__variance,PDO__maximum,PDO__mean,PDO__median,PDO__minimum,PDO__variance,PTS_expected__maximum,PTS_expected__mean,PTS_expected__median,PTS_expected__minimum,PTS_expected__variance,SCF%__maximum,SCF%__mean,SCF%__median,SCF%__minimum,SCF%__variance,SCF/60__maximum,SCF/60__mean,SCF/60__median,SCF/60__minimum,SCF/60__variance,SF%__maximum,SF%__mean,SF%__median,SF%__minimum,SF%__variance,SF/60__maximum,SF/60__mean,SF/60__median,SF/60__minimum,SF/60__variance,SH%_PP__maximum,SH%_PP__mean,SH%_PP__median,SH%_PP__minimum,SH%_PP__variance,Shots/60__maximum,Shots/60__mean,Shots/60__median,Shots/60__minimum,Shots/60__variance,Shots_PP__maximum,Shots_PP__mean,Shots_PP__median,Shots_PP__minimum,Shots_PP__variance,TOI/GP_x__maximum,TOI/GP_x__mean,TOI/GP_x__median,TOI/GP_x__minimum,TOI/GP_x__variance,TOI_PP__maximum,TOI_PP__mean,TOI_PP__median,TOI_PP__minimum,TOI_PP__variance,TOI__maximum,TOI__mean,TOI__median,TOI__minimum,TOI__variance,Total Points/60__maximum,Total Points/60__mean,Total Points/60__median,Total Points/60__minimum,Total Points/60__variance,Total Points/GP__maximum,Total Points/GP__mean,Total Points/GP__median,Total Points/GP__minimum,Total Points/GP__variance,Total Points_PP__maximum,Total Points_PP__mean,Total Points_PP__median,Total Points_PP__minimum,Total Points_PP__variance,Total Points__maximum,Total Points__mean,Total Points__median,Total Points__minimum,Total Points__variance,iCF/60__maximum,iCF/60__mean,iCF/60__median,iCF/60__minimum,iCF/60__variance,iCF_PP__maximum,iCF_PP__mean,iCF_PP__median,iCF_PP__minimum,iCF_PP__variance,iFF/60__maximum,iFF/60__mean,iFF/60__median,iFF/60__minimum,iFF/60__variance,iFF_PP__maximum,iFF_PP__mean,iFF_PP__median,iFF_PP__minimum,iFF_PP__variance,iHDCF/60__maximum,iHDCF/60__mean,iHDCF/60__median,iHDCF/60__minimum,iHDCF/60__variance,iHDCF_PP__maximum,iHDCF_PP__mean,iHDCF_PP__median,iHDCF_PP__minimum,iHDCF_PP__variance,iSCF/60__maximum,iSCF/60__mean,iSCF/60__median,iSCF/60__minimum,iSCF/60__variance,iSCF_PP__maximum,iSCF_PP__mean,iSCF_PP__median,iSCF_PP__minimum,iSCF_PP__variance,ixG/60__maximum,ixG/60__mean,ixG/60__median,ixG/60__minimum,ixG/60__variance,ixG_PP__maximum,ixG_PP__mean,ixG_PP__median,ixG_PP__minimum,ixG_PP__variance,xGA/60__maximum,xGA/60__mean,xGA/60__median,xGA/60__minimum,xGA/60__variance,xGF%__maximum,xGF%__mean,xGF%__median,xGF%__minimum,xGF%__variance,xGF/60__maximum,xGF/60__mean,xGF/60__median,xGF/60__minimum,xGF/60__variance,Age,Overall Draft Position,Age_2014,Age_2015,Age_2016,Age_2017,Age_2018,CF%_2014,CF%_2015,CF%_2016,CF%_2017,CF%_2018,CF/60_2014,CF/60_2015,CF/60_2016,CF/60_2017,CF/60_2018,FF%_2014,FF%_2015,FF%_2016,FF%_2017,FF%_2018,FF/60_2014,FF/60_2015,FF/60_2016,FF/60_2017,FF/60_2018,GF%_2014,GF%_2015,GF%_2016,GF%_2017,GF%_2018,GF/60_2014,GF/60_2015,GF/60_2016,GF/60_2017,GF/60_2018,GP_2014,

## 4. Contracts
---

1. Export the most recent pool spreadsheet and save it to z../data/Pool-Att-1819.csvz, for example, for forwards of the NHL season 2018-2019. It is necessary to get contract ends to avoid rewriting them.

**TODO: Use capfriendly contract tables?**

### 4.1 Forwards

In [19]:
nhl_att_contracts = pd.read_csv('../data/Pool-Att-1819.csv', header=1, encoding = "utf-8")
# TODO: Cap Hit_18 will be different probably
nhl_att_contracts = nhl_att_contracts.rename(columns={'Unnamed: 95': "Cap Hit_18", 'Unnamed: 96': "END", "Unnamed: 0": "Name"})
nhl_att_contracts['Name'] = nhl_att_contracts.apply(lambda player: re.sub(r'\W+', '', f'{player["Unnamed: 3"]}{player["Unnamed: 2"]}').upper(), axis=1)
nhl_att_contracts.set_index('Name', inplace=True)
nhl_att_contracts.rename({'BURMISTROVALEX': 'BURMISTROVALEXANDER',
                          'BURROWSALEX': 'BURROWSALEXANDRE',
                          'CAMMALLERIMIKE': 'CAMMALLERIMICHAEL',
                          'DEANGELOTONY': 'DEANGELOANTHONY',
                          'DIDOMENICOCHRIS': 'DIDOMENICOCHRISTOPHER',
                          'EDLERALEX': 'EDLERALEXANDER',
                          'ENSTROMTOBY': 'ENSTROMTOBIAS',
                          'FERLANDMICHEAL': 'FERLANDMICHAEL',
                          'HINOSTROZAVINNIE': 'HINOSTROZAVINCENT',
                          'KULEMINNIKOLAY': 'KULEMINNIKOLAI',
                          'MARCHESSAULTJONATHAN': 'AUDYMARCHESSAULTJONATHAN',
                          'MATHESONMIKE': 'MATHESONMICHAEL',
                          'MORRISSEYJOSH': 'MORRISSEYJOSHUA',
                          'PETROVICALEX': 'PETROVICALEXANDER',
                          'RADULOVALEX': 'RADULOVALEXANDER',
                          'WENNBERGALEX': 'WENNBERGALEXANDER'}, inplace = True)
nhl_att_contracts.head()

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Position,GP_13,GP_14,GP_15,GP_16,GP_17,Real,Mean,Var,G_13,G_14,G_15,G_16,G_17,G_18,A_13,A_14,A_15,A_16,A_17,A_18,PTS_13,PTS_14,PTS_15,PTS_16,PTS_17,Proj,Chuck Lemieux,NHL,ESPN,Fantrax,TSN,Real.1,Mean.1,Var.1,PAX,PTS/G_13,PTS/G_14,PTS/G_15,PTS/G_16,PTS/G_17,Real.2,Mean.2,Var.2,$/PTS_13,$/PTS_14,$/PTS_15,$/PTS_16,$/PTS_17,Unnamed: 56,Sh%_13,Sh%_14,Sh%_15,Sh%_16,Sh%_17,iSF_13,iSF_14,iSF_15,iSF_16,iSF_17,iSF,TOI/G_13,TOI/G_14,TOI/G_15,TOI/G_16,TOI/G_17,5v5,PPTOI/GP_17,PP,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Cap Hit_18,END,Unnamed: 97,Unnamed: 98,Unnamed: 99
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
CONNORMCDAVID,McDavid Connor,McDavid,Connor,X,EDM,22.0,C,NaN,NaN,NaN,45.0,82.0,82.0,70.0,304.0,NaN,NaN,NaN,16.0,30.0,41.0,NaN,NaN,NaN,32.0,70.0,67.0,NaN,NaN,NaN,48.0,100.0,108.0,112.0,105.0,111.0,105.0,108.0,108.0,85.0,708.0,12.0,NaN,NaN,NaN,1.066667,1.219512,1.32,1.20,0.011,NaN,NaN,NaN,10575.45732,9250.00000,"115,527",NaN,NaN,NaN,0.152,0.124,NaN,NaN,NaN,105.0,252.0,275.0,NaN,NaN,NaN,18.9,21.13,21.55,2.87,2.97,10.4%,61%,40.6,1184.0,1.32,7.86,10.5,1032,1004.0,421.0,434.0,329.0,357.0,NaN,NaN,NaN,925000.0,925000.0,925000.0,"12,500,000",26.0,18.14,9.70,NaN
NIKITAKUCHEROV,Kucherov Nikita,Kucherov,Nikita,X,TBL,25.0,RW,NaN,52.0,82.0,77.0,74.0,80.0,73.0,118.0,NaN,9.0,29.0,30.0,40.0,39.0,NaN,9.0,36.0,36.0,45.0,61.0,NaN,18.0,65.0,66.0,85.0,97.0,106.0,95.0,94.0,97.0,92.0,100.0,67.0,764.0,34.0,NaN,0.346154,0.792683,0.857143,1.148649,1.25,0.88,0.100,NaN,31501.81843,13676.92308,10130.08130,50607.36844,"49,242",NaN,0.088,0.152,0.144,0.120,NaN,102.0,191.0,209.0,246.0,279.0,NaN,13.1,15.0,18.2,19.43,19.82,3.20,3.51,11.9%,71%,37.4,1036.0,1.27,5.29,9.8,1032,1035.0,464.0,546.0,348.0,422.0,NaN,894167.0,889000.0,712000.0,4766667.0,4766667.0,"4,766,667",19.0,14.50,9.18,NaN
SIDNEYCROSBY,Crosby Sidney,Crosby,Sidney,X,PIT,31.0,C,36.0,80.0,77.0,80.0,75.0,82.0,79.0,6.0,15.0,36.0,28.0,36.0,44.0,29.0,41.0,68.0,56.0,49.0,45.0,60.0,56.0,104.0,84.0,85.0,89.0,91.0,90.0,92.0,88.0,90.0,94.0,89.0,90.0,52.0,-19.0,1.555556,1.300000,1.090909,1.062500,1.186667,1.09,1.21,0.030,68205.57491,81613.50844,97256.09756,99856.52798,89408.05700,"95,815",0.121,0.139,0.118,0.145,0.114,124.0,259.0,237.0,248.0,255.0,247.0,21.1,22.0,20.0,20.5,19.88,20.69,3.47,3.65,10.8%,66%,32.1,1215.0,1.14,3.09,7.3,1031,1008.0,411.0,412.0,325.0,341.0,8700000.0,8700000.0,8700000.0,8700000.0,8700000.0,8700000.0,"8,700,000",25.0,12.68,9.43,NaN
NATHANMACKINNON,MacKinnon Nathan,MacKinnon,Nathan,X,COL,23.0,C,NaN,82.0,64.0,72.0,82.0,74.0,75.0,46.0,NaN,24.0,14.0,21.0,16.0,39.0,NaN,39.0,24.0,31.0,37.0,58.0,NaN,63.0,38.0,52.0,53.0,90.0,95.0,90.0,89.0,86.0,88.0,97.0,61.0,395.0,46.0,NaN,0.768293,0.593750,0.722222,0.646341,1.31,0.81,0.067,NaN,59920.63492,77535.30167,15619.13696,118867.92450,"70,313",NaN,0.100,0.073,0.086,0.088,NaN,241.0,192.0,245.0,252.0,284.0,NaN,17.3,17.0,18.9,19.95,19.91,2.63,3.61,13.8%,65%,37.0,885.0,1.25,5.08,10.0,981,1039.0,436.0,496.0,333.0,393.0,NaN,3775000.0,3775000.0,925000.0,6300000.0,6300000.0,"6,300,000",23.0,17.20,9.08,NaN
JOHNNYGAUDREAU,Gaudreau Johnny,Gaudreau,Johnny,X,CGY,25.0,LW,NaN,1.0,80.0,79.0,72.0,80.0,62.0,951.0,NaN,1.0,24.0,30.0,18.0,24.0,NaN,0.0,40.0,48.0,43.0,60.0,NaN,1.0,64.0,78.0,61.0,86.0,88.0,86.0,85.0,88.0,81.0,84.0,58.0,874.0,14.0,NaN,1.000000,0.800000,0.987342,0.847222,1.05,0.94,0.009,NaN,22560.97561,28201.21951,11425.10944,97161.13555,"78,855",NaN,1.000,0.144,0.138,0.097,NaN,1.0,167.0,217.0,182.0,227.0,NaN,15.2,17.7,19.9,18.48,19.42,3.09,3.54,10.4%,69%,29.4,1092.0,1.10,1.14,6.7,1006,999.0,307.0,407.0,241.0,316.0,NaN,1850000.0,1850000.0,925000.0,6750000.0,6750000.0,"6,750,000",22.0,8.42,7.27,NaN


### 4.2 Defensemen

In [20]:
nhl_def_contracts = pd.read_csv('../data/Pool-Def-1819.csv', header=1, encoding = "utf-8")
# TODO: Cap Hit_18 will be different probably
nhl_def_contracts = nhl_def_contracts.rename(columns={'Unnamed: 95': "Cap Hit_18", 'Unnamed: 96': "END", "Unnamed: 0": "Name"})
nhl_def_contracts['Name'] = nhl_def_contracts.apply(lambda player: re.sub(r'\W+', '', f'{player["Unnamed: 3"]}{player["Unnamed: 2"]}').upper(), axis=1)
nhl_def_contracts.set_index('Name', inplace=True)
nhl_def_contracts.rename({'BURMISTROVALEX': 'BURMISTROVALEXANDER',
           'BURROWSALEX': 'BURROWSALEXANDRE',
           'CAMMALLERIMIKE': 'CAMMALLERIMICHAEL',
           'DEANGELOTONY': 'DEANGELOANTHONY',
           'DIDOMENICOCHRIS': 'DIDOMENICOCHRISTOPHER',
           'EDLERALEX': 'EDLERALEXANDER',
           'ENSTROMTOBY': 'ENSTROMTOBIAS',
           'FERLANDMICHEAL': 'FERLANDMICHAEL',
           'HINOSTROZAVINNIE': 'HINOSTROZAVINCENT',
           'KULEMINNIKOLAY': 'KULEMINNIKOLAI',
           'MARCHESSAULTJONATHAN': 'AUDYMARCHESSAULTJONATHAN',
           'MATHESONMIKE': 'MATHESONMICHAEL',
           'MORRISSEYJOSH': 'MORRISSEYJOSHUA',
           'PETROVICALEX': 'PETROVICALEXANDER',
           'RADULOVALEX': 'RADULOVALEXANDER',
           'WENNBERGALEX': 'WENNBERGALEXANDER'}, inplace = True)
nhl_def_contracts.head()

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Position,GP_13,GP_14,GP_15,GP_16,GP_17,Real,Mean,Var,G_13,G_14,G_15,G_16,G_17,G_18,A_13,A_14,A_15,A_16,A_17,A_18,PTS_13,PTS_14,PTS_15,PTS_16,PTS_17,Proj,Chuck Lemieux,NHL,ESPN,Fantrax,TSN,Real.1,Mean.1,Var.1,PAX,PTS/G_13,PTS/G_14,PTS/G_15,PTS/G_16,PTS/G_17,Real.2,Mean.2,Var.2,$/PTS_13,$/PTS_14,$/PTS_15,$/PTS_16,$/PTS_17,Unnamed: 56,Sh%_13,Sh%_14,Sh%_15,Sh%_16,Sh%_17,iSF_13,iSF_14,iSF_15,iSF_16,iSF_17,iSF,TOI/G_13,TOI/G_14,TOI/G_15,TOI/G_16,TOI/G_17,5v5,PPTOI/GP_17,PP,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Cap Hit_18,END,Unnamed: 97,Unnamed: 98,Unnamed: 99
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
BRENTBURNS,Burns Brent,Burns,Brent,X,SJS,33.0,D,30.0,69.0,82.0,82.0,82.0,82.0,79.0,27.0,9.0,22.0,17.0,27.0,29.0,12.0,11.0,26.0,43.0,48.0,47.0,55.0,20.0,48.0,60.0,75.0,76.0,70.0,70.0,71.0,75.0,64.0,71.0,67.0,65.0,108.0,20.0,0.666667,0.695652,0.731707,0.914634,0.926829,0.82,0.79,0.010,105365.8537,100975.60980,96000.000000,76800.00000,75789.47368,"113,960",0.111,0.090,0.069,0.076,0.099,81.0,245.0,245.0,353.0,322.0,332.0,16.3,16.8,24.0,25.9,24.93,25.24,3.36,3.68,8.5%,67%,21.6,1245.0,1.02,0.39,5.4,1019.0,1006.0,777.0,774.0,483.0,481.0,5760000.0,5760000.0,5760000.0,5760000.0,5760000.0,8000000.0,"8,000,000",25.0,6.49,8.96,NaN
ERIKKARLSSON,Karlsson Erik,Karlsson,Erik,X,OTT,28.0,D,17.0,82.0,82.0,82.0,77.0,71.0,79.0,19.0,6.0,20.0,21.0,16.0,17.0,9.0,8.0,54.0,45.0,66.0,54.0,53.0,14.0,74.0,66.0,82.0,71.0,68.0,69.0,72.0,73.0,58.0,70.0,62.0,71.0,47.0,4.0,0.823529,0.902439,0.804878,1.000000,0.922078,0.87,0.89,0.004,96254.3554,87837.83784,98484.848480,79268.29268,85967.02164,"95,029",0.076,0.078,0.072,0.065,0.095,79.0,257.0,292.0,248.0,218.0,196.0,27.2,27.1,27.3,29.0,26.83,26.74,3.55,3.32,9.2%,61%,20.1,951.0,0.85,0.86,4.9,1013.0,1033.0,506.0,477.0,337.0,296.0,6500000.0,6500000.0,6500000.0,6500000.0,6500000.0,6500000.0,"6,500,000",19.0,6.36,7.96,NaN
VICTORHEDMAN,Hedman Victor,Hedman,Victor,X,TBL,28.0,D,44.0,75.0,59.0,78.0,79.0,77.0,74.0,55.0,4.0,13.0,10.0,10.0,16.0,17.0,16.0,42.0,28.0,37.0,56.0,46.0,20.0,55.0,38.0,47.0,72.0,65.0,68.0,68.0,66.0,62.0,62.0,63.0,55.0,141.0,21.0,0.454545,0.733333,0.644068,0.602564,0.911392,0.82,0.69,0.022,107317.0732,66518.84701,75738.125800,80954.85210,53523.03523,"120,782",0.053,0.076,0.087,0.056,0.118,76.0,170.0,115.0,180.0,166.0,216.0,22.7,22.4,22.7,23.1,24.50,25.85,2.95,3.40,11.6%,54%,22.0,1067.0,0.93,0.04,5.7,1031.0,971.0,408.0,469.0,270.0,312.0,4000000.0,4000000.0,4000000.0,4000000.0,4000000.0,7875000.0,"7,875,000",25.0,6.64,7.54,NaN
JOHNKLINGBERG,Klingberg John,Klingberg,John,X,DAL,26.0,D,NaN,NaN,65.0,76.0,80.0,82.0,76.0,43.0,NaN,NaN,11.0,10.0,13.0,8.0,NaN,NaN,29.0,48.0,36.0,59.0,NaN,NaN,40.0,58.0,49.0,63.0,71.0,66.0,63.0,54.0,61.0,67.0,54.0,101.0,13.0,NaN,NaN,0.615385,0.763158,0.612500,0.82,0.70,0.008,NaN,NaN,14268.292680,67914.21362,84619.21354,"67,460",NaN,NaN,0.112,0.058,0.106,NaN,NaN,98.0,171.0,124.0,204.0,NaN,NaN,21.8,22.7,23.35,24.07,3.03,3.65,10.2%,62%,22.1,1226.0,0.91,1.66,5.4,1016.0,983.0,313.0,455.0,194.0,286.0,NaN,NaN,720000.0,4250000.0,4250000.0,4250000.0,"4,250,000",22.0,5.82,8.00,NaN
SHAYNEGOSTISBEHERE,Gostisbehere Shayne,Gostisbehere,Shayne,X,PHI,25.0,D,NaN,NaN,2.0,64.0,76.0,78.0,55.0,965.0,NaN,NaN,0.0,17.0,7.0,13.0,NaN,NaN,0.0,29.0,32.0,52.0,NaN,NaN,0.0,46.0,39.0,61.0,67.0,57.0,69.0,57.0,56.0,65.0,38.0,559.0,18.0,NaN,NaN,0.000000,0.718750,0.513158,0.83,0.52,0.102,NaN,NaN,1292.682927,15694.59173,21982.48906,"73,529",NaN,NaN,0.000,0.112,0.082,NaN,NaN,2.0,152.0,198.0,221.0,NaN,NaN,12.6,20.1,19.58,21.46,3.65,3.61,11.3%,68%,21.1,972.0,0.90,0.11,5.8,992.0,975.0,445.0,458.0,303.0,320.0,NaN,NaN,53000.0,925000.0,925000.0,4500000.0,"4,500,000",23.0,4.85,6.25,NaN


### 4.3 Merge All

In [21]:
nhl_contracts = pd.concat([nhl_att_contracts, nhl_def_contracts], sort=True)
# TODO: Cap Hit_18 will be different probably
nhl = nhl.join(nhl_contracts[['Unnamed: 2', 'Unnamed: 3', 'END', 'Cap Hit_18']])
nhl = nhl.rename({'Unnamed: 2': "Last Name", 'Unnamed: 3': 'First Name'}, axis=1)

## 5. Salary Correction
---

1. Update with salaries from 2019-2020 season from https://www.hockey-reference.com/friv/current_nhl_salaries.cgi

2. Save table under `../data/Salaries-2019-2020.csv` for NHL season 2019-2020 for example.

**TODO: Use capfriendly salary tables?**

In [22]:
# Salaries of 2019-2020 season
nhl_salaries = pd.read_csv('../data/Salaries-2019-2020.csv', header=None, sep=';', encoding = "utf-8")
nhl_salaries.columns = ['Player', 'Team', 'Salary', 'Cap_Hit']
nhl_salaries['Name'] = nhl_salaries.Player.apply(lambda player: re.sub(r'\W+', '', f'{player}').upper())
nhl_salaries.set_index('Name', inplace=True)
nhl_salaries.rename({'BURMISTROVALEX': 'BURMISTROVALEXANDER',
                     'BURROWSALEX': 'BURROWSALEXANDRE',
                     'CAMMALLERIMIKE': 'CAMMALLERIMICHAEL',
                     'DEANGELOTONY': 'DEANGELOANTHONY',
                     'DIDOMENICOCHRIS': 'DIDOMENICOCHRISTOPHER',
                     'EDLERALEX': 'EDLERALEXANDER',
                     'ENSTROMTOBY': 'ENSTROMTOBIAS',
                     'FERLANDMICHEAL': 'FERLANDMICHAEL',
                     'HINOSTROZAVINNIE': 'HINOSTROZAVINCENT',
                     'KULEMINNIKOLAY': 'KULEMINNIKOLAI',
                     'MARCHESSAULTJONATHAN': 'AUDYMARCHESSAULTJONATHAN',
                     'MATHESONMIKE': 'MATHESONMICHAEL',
                     'MORRISSEYJOSH': 'MORRISSEYJOSHUA',
                     'PETROVICALEX': 'PETROVICALEXANDER',
                     'RADULOVALEX': 'RADULOVALEXANDER',
                     'WENNBERGALEX': 'WENNBERGALEXANDER'}, inplace = True)
nhl_salaries.head()

# Reverse index of df and join the salaries
nhl = nhl.drop('Team', axis=1)
nhl = nhl.join(nhl_salaries[['Team', 'Cap_Hit']])

# Convert to float
nhl['Cap_Hit'] = nhl['Cap_Hit'].replace(',', '', regex=True).astype(float)

# If contract ended last year, put ending year and cap hit from last year to NaN and add manually
# TODO: END year will be different and Cap Hit_18
nhl.loc[nhl.END == 19, 'Cap Hit_18'] = np.nan
nhl.loc[nhl.END == 19, 'END'] = np.nan

# If no match for hockey reference salaries, use contract from last year
# All contracts ending have been assigne NaN, so all remaining contracts are valid, 
# thus, this operation will transfer only good contracts
# TODO: Cap Hit_18 will be different
nhl.loc[~(nhl.Cap_Hit > 0), 'Cap_Hit'] = nhl.loc[~(nhl.Cap_Hit > 0), 'Cap Hit_18']
nhl = nhl.drop('Cap Hit_18', axis=1)
nhl.head()

,Player,Team,Salary,Cap_Hit
Name,,,,
JOHNTAVARES,John Tavares,TOR,"15,900,000","11,000,000"
AUSTONMATTHEWS,Auston Matthews,TOR,"15,900,000","11,634,000"
CAREYPRICE,Carey Price,MTL,"15,000,000","10,500,000"
ERIKKARLSSON,Erik Karlsson,SJS,"14,500,000","11,500,000"
ARTEMIPANARIN,Artemi Panarin,NYR,"14,000,000","11,642,857"


,CF%__maximum,CF%__mean,CF%__median,CF%__minimum,CF%__variance,CF/60__maximum,CF/60__mean,CF/60__median,CF/60__minimum,CF/60__variance,FF%__maximum,FF%__mean,FF%__median,FF%__minimum,FF%__variance,FF/60__maximum,FF/60__mean,FF/60__median,FF/60__minimum,FF/60__variance,GF%__maximum,GF%__mean,GF%__median,GF%__minimum,GF%__variance,GF/60__maximum,GF/60__mean,GF/60__median,GF/60__minimum,GF/60__variance,GP__maximum,GP__mean,GP__median,GP__minimum,GP__variance,IPP_PP__maximum,IPP_PP__mean,IPP_PP__median,IPP_PP__minimum,IPP_PP__variance,Off. Zone Faceoff %__maximum,Off. Zone Faceoff %__mean,Off. Zone Faceoff %__median,Off. Zone Faceoff %__minimum,Off. Zone Faceoff %__variance,Off. Zone Faceoffs/60__maximum,Off. Zone Faceoffs/60__mean,Off. Zone Faceoffs/60__median,Off. Zone Faceoffs/60__minimum,Off. Zone Faceoffs/60__variance,Off. Zone Start %__maximum,Off. Zone Start %__mean,Off. Zone Start %__median,Off. Zone Start %__minimum,Off. Zone Start %__variance,Off. Zone Starts/60__maximum,Off. Zone Starts/60__mean,Off. Zone Starts/60__median,Off. Zone Starts/60__minimum,Off. Zone Starts/60__variance,On-Ice SH%__maximum,On-Ice SH%__mean,On-Ice SH%__median,On-Ice SH%__minimum,On-Ice SH%__variance,PDO__maximum,PDO__mean,PDO__median,PDO__minimum,PDO__variance,PTS_expected__maximum,PTS_expected__mean,PTS_expected__median,PTS_expected__minimum,PTS_expected__variance,SCF%__maximum,SCF%__mean,SCF%__median,SCF%__minimum,SCF%__variance,SCF/60__maximum,SCF/60__mean,SCF/60__median,SCF/60__minimum,SCF/60__variance,SF%__maximum,SF%__mean,SF%__median,SF%__minimum,SF%__variance,SF/60__maximum,SF/60__mean,SF/60__median,SF/60__minimum,SF/60__variance,SH%_PP__maximum,SH%_PP__mean,SH%_PP__median,SH%_PP__minimum,SH%_PP__variance,Shots/60__maximum,Shots/60__mean,Shots/60__median,Shots/60__minimum,Shots/60__variance,Shots_PP__maximum,Shots_PP__mean,Shots_PP__median,Shots_PP__minimum,Shots_PP__variance,TOI/GP_x__maximum,TOI/GP_x__mean,TOI/GP_x__median,TOI/GP_x__minimum,TOI/GP_x__variance,TOI_PP__maximum,TOI_PP__mean,TOI_PP__median,TOI_PP__minimum,TOI_PP__variance,TOI__maximum,TOI__mean,TOI__median,TOI__minimum,TOI__variance,Total Points/60__maximum,Total Points/60__mean,Total Points/60__median,Total Points/60__minimum,Total Points/60__variance,Total Points/GP__maximum,Total Points/GP__mean,Total Points/GP__median,Total Points/GP__minimum,Total Points/GP__variance,Total Points_PP__maximum,Total Points_PP__mean,Total Points_PP__median,Total Points_PP__minimum,Total Points_PP__variance,Total Points__maximum,Total Points__mean,Total Points__median,Total Points__minimum,Total Points__variance,iCF/60__maximum,iCF/60__mean,iCF/60__median,iCF/60__minimum,iCF/60__variance,iCF_PP__maximum,iCF_PP__mean,iCF_PP__median,iCF_PP__minimum,iCF_PP__variance,iFF/60__maximum,iFF/60__mean,iFF/60__median,iFF/60__minimum,iFF/60__variance,iFF_PP__maximum,iFF_PP__mean,iFF_PP__median,iFF_PP__minimum,iFF_PP__variance,iHDCF/60__maximum,iHDCF/60__mean,iHDCF/60__median,iHDCF/60__minimum,iHDCF/60__variance,iHDCF_PP__maximum,iHDCF_PP__mean,iHDCF_PP__median,iHDCF_PP__minimum,iHDCF_PP__variance,iSCF/60__maximum,iSCF/60__mean,iSCF/60__median,iSCF/60__minimum,iSCF/60__variance,iSCF_PP__maximum,iSCF_PP__mean,iSCF_PP__median,iSCF_PP__minimum,iSCF_PP__variance,ixG/60__maximum,ixG/60__mean,ixG/60__median,ixG/60__minimum,ixG/60__variance,ixG_PP__maximum,ixG_PP__mean,ixG_PP__median,ixG_PP__minimum,ixG_PP__variance,xGA/60__maximum,xGA/60__mean,xGA/60__median,xGA/60__minimum,xGA/60__variance,xGF%__maximum,xGF%__mean,xGF%__median,xGF%__minimum,xGF%__variance,xGF/60__maximum,xGF/60__mean,xGF/60__median,xGF/60__minimum,xGF/60__variance,Age,Overall Draft Position,Age_2014,Age_2015,Age_2016,Age_2017,Age_2018,CF%_2014,CF%_2015,CF%_2016,CF%_2017,CF%_2018,CF/60_2014,CF/60_2015,CF/60_2016,CF/60_2017,CF/60_2018,FF%_2014,FF%_2015,FF%_2016,FF%_2017,FF%_2018,FF/60_2014,FF/60_2015,FF/60_2016,FF/60_2017,FF/60_2018,GF%_2014,GF%_2015,GF%_2016,GF%_2017,GF%_2018,GF/60_2014,GF/60_2015,GF/60_2016,GF/60_2017,GF/60_2018,GP_2014,

In [23]:
# Filter to get only desired columns
nhl = nhl.filter(regex=("First Name|Last Name|Age$|Position|Team|^GP|^TOI/GP_x|^Total Points|^PTS_expected|^Total Points/GP|^On-Ice SH%|^TOI_PP|^Total Points_PP|^iSCF|^Off. Zone Start %|^PDO|Score|END|Cap_Hit"))
nhl['Team'] = nhl['Team'].apply(lambda tm: str(tm).split('/')[-1])
# TODO: 'TOI/GP_PP_2018' will be different
nhl['TOI/GP_PP_2018'] = nhl['TOI_PP_2018']/nhl['GP_2018']
nhl = nhl[nhl.index != 'NANNAN']
nhl.head(10)

,GP__maximum,GP__mean,GP__median,GP__minimum,GP__variance,Off. Zone Start %__maximum,Off. Zone Start %__mean,Off. Zone Start %__median,Off. Zone Start %__minimum,Off. Zone Start %__variance,On-Ice SH%__maximum,On-Ice SH%__mean,On-Ice SH%__median,On-Ice SH%__minimum,On-Ice SH%__variance,PDO__maximum,PDO__mean,PDO__median,PDO__minimum,PDO__variance,PTS_expected__maximum,PTS_expected__mean,PTS_expected__median,PTS_expected__minimum,PTS_expected__variance,TOI/GP_x__maximum,TOI/GP_x__mean,TOI/GP_x__median,TOI/GP_x__minimum,TOI/GP_x__variance,TOI_PP__maximum,TOI_PP__mean,TOI_PP__median,TOI_PP__minimum,TOI_PP__variance,Total Points/60__maximum,Total Points/60__mean,Total Points/60__median,Total Points/60__minimum,Total Points/60__variance,Total Points/GP__maximum,Total Points/GP__mean,Total Points/GP__median,Total Points/GP__minimum,Total Points/GP__variance,Total Points_PP__maximum,Total Points_PP__mean,Total Points_PP__median,Total Points_PP__minimum,Total Points_PP__variance,Total Points__maximum,Total Points__mean,Total Points__median,Total Points__minimum,Total Points__variance,iSCF/60__maximum,iSCF/60__mean,iSCF/60__median,iSCF/60__minimum,iSCF/60__variance,iSCF_PP__maximum,iSCF_PP__mean,iSCF_PP__median,iSCF_PP__minimum,iSCF_PP__variance,Age,Overall Draft Position,GP_2014,GP_2015,GP_2016,GP_2017,GP_2018,Off. Zone Start %_2014,Off. Zone Start %_2015,Off. Zone Start %_2016,Off. Zone Start %_2017,Off. Zone Start %_2018,On-Ice SH%_2014,On-Ice SH%_2015,On-Ice SH%_2016,On-Ice SH%_2017,On-Ice SH%_2018,PDO_2014,PDO_2015,PDO_2016,PDO_2017,PDO_2018,PTS_expected_2014,PTS_expected_2015,PTS_expected_2016,PTS_expected_2017,PTS_expected_2018,TOI/GP_x_2014,TOI/GP_x_2015,TOI/GP_x_2016,TOI/GP_x_2017,TOI/GP_x_2018,TOI_PP_2014,TOI_PP_2015,TOI_PP_2016,TOI_PP_2017,TOI_PP_2018,Total Points_2014,Total Points_2015,Total Points_2016,Total Points_2017,Total Points_2018,Total Points/60_2014,Total Points/60_2015,Total Points/60_2016,Total Points/60_2017,Total Points/60_2018,Total Points/GP_2014,Total Points/GP_2015,Total Points/GP_2016,Total Points/GP_2017,Total Points/GP_2018,Total Points_PP_2014,Total Points_PP_2015,Total Points_PP_2016,Total Points_PP_2017,Total Points_PP_2018,iSCF/60_2014,iSCF/60_2015,iSCF/60_2016,iSCF/60_2017,iSCF/60_2018,iSCF_PP_2014,iSCF_PP_2015,iSCF_PP_2016,iSCF_PP_2017,iSCF_PP_2018,Position,Score,Last Name,First Name,END,Team,Cap_Hit,TOI/GP_PP_2018
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AARONEKBLAD,82.0,78.2,81.0,68.0,28.160000,68.88,55.934000,60.18,40.09,154.077664,10.57,9.200000,10.32,6.24,2.780920,1.019,1.000400,1.006,0.974,0.000324,38.0,35.000000,38.0,24.0,30.400000,23.644106,22.396080,21.810082,21.460049,0.855987,226.550000,190.563333,190.466667,138.800000,1032.308489,1.32,1.160000,1.190,0.86,0.026200,0.469136,0.427885,0.461538,0.294118,0.004507,13.0,10.200000,10.0,8.0,2.960000,38.0,33.800000,37.0,20.0,48.160000,4.56,3.360000,3.05,2.65,0.492280,32.0,22.0,20.0,15.0,33.200000,22,1,81.0,78.0,68.0,82.0,82.0,68.88,60.18,68.16,42.36,40.09,8.46,10.57,6.24,10.32,10.41,1.006,1.019,0.974,1.018,0.985,38.0,38.0,24.0,38.0,37.0,21.810082,21.677137,21.460049,23.389024,23.644106,226.550000,221.533333,190.466667,175.466667,138.800000,38.0,36.0,20.0,38.0,37.0,1.32,1.28,0.86,1.19,1.15,0.469136,0.461538,0.294118,0.463415,0.451220,13.0,9.0,8.0,11.0,10.0,2.65,3.05,4.56,3.72,2.82,19.0,20.0,32.0,24.0,15.0,D,6.771978,Ekblad,Aaron,25.0,FLA,7.5e+06,1.692683
AARONNESS,8.0,8.0,8.0,8.0,0.000000,81.25,81.250000,81.25,81.25,0.000000,13.33,13.330000,13.33,13.33,0.000000,1.082,1.082000,1.082,1.082,0.000000,10.0,10.000000,10.0,10.0,0.000000,12.387500,12.387500,12.387500,12.387500,0.000000,0.150000,0.150000,0.150000,0.150000,0.000000,1.21,1.210000,1.210,1.21,0.000000,0.125000,0.125000,0.125000,0.125000,0.000000,0.0,0.000000,0.0,0.0,0.000000,1.0,1.000000,1.0,1.0,0.000000,1.82,1.820000,1.82,1.82,0.000000,0.0,0.0,0.0,0.0,0.000000,25,40,NaN,8.0,NaN,NaN,NaN,NaN,

## 6. Model Predictions
---


### 6.1 LSTM

In [24]:
lstm_preds = []
# TODO: Make sure predictions are updated
for player in glob.glob('../LSTM-Neural-Network-for-Time-Series-Prediction/predictions/*.pkl'):
    tmp = pd.read_pickle(player)
    tmp = pd.DataFrame.from_dict(tmp, orient='index')
    lstm_preds.append(tmp)
lstm_preds = pd.concat(lstm_preds)
lstm_preds['Player_Id'] = lstm_preds.apply(lambda x: x.name.split('.cs')[0].split('-')[-1], axis=1)
lstm_preds['LSTM_full'] = lstm_preds.apply(lambda x: np.array(x['full-sequence']).sum(), axis=1)
lstm_preds['LSTM_point'] = lstm_preds.apply(lambda x: np.array(x['point-by-point']).sum(), axis=1)
lstm_preds.index.name = 'name'
lstm_preds.head()

,multiple-sequences,full-sequence,point-by-point,train,test,Player_Id,LSTM_full,LSTM_point
name,,,,,,,,
MARTIN-BRODEUR-8455710.csv,"[[0.10443657, 0.10036451, 0.09233941, 0.082081...","[0.10443657, 0.10036451, 0.09233941, 0.0820819...","[0.10443657, 0.10443657, 0.10443657, 0.1044365...","[2013-10-04 00:00:00, 2014-12-29 00:00:00]","[2015-01-02 00:00:00, 2015-01-02 00:00:00]",8455710,6.465791,8.563799
MATT-BELESKEY-8473492.csv,"[[0.29253426, 0.29245296, 0.29218978, 0.291729...","[0.29253426, 0.29245296, 0.29218978, 0.2917294...","[0.29253426, 0.29253426, 0.29253426, 0.2925342...","[2013-10-02 00:00:00, 2018-12-10 00:00:00]","[2018-12-14 00:00:00, 2018-12-14 00:00:00]",8473492,23.875305,23.987810
JANNIK-HANSEN-8471498.csv,"[[0.37088394, 0.3701646, 0.36744553, 0.3628682...","[0.37088394, 0.3701646, 0.36744553, 0.36286822...","[0.37088388, 0.37088388, 0.37088388, 0.3708838...","[2013-10-03 00:00:00, 2018-03-29 00:00:00]","[2018-04-05 00:00:00, 2018-04-05 00:00:00]",8471498,29.364569,30.412481
NIC-PETAN-8477464.csv,"[[0.10454964, 0.10764119, 0.115878664, 0.13068...","[0.10454964, 0.10764119, 0.115878664, 0.130686...","[0.10454964, 0.10454964, 0.10454964, 0.1045496...","[2015-10-08 00:00:00, 2019-03-19 00:00:00]","[2019-03-20 00:00:00, 2019-03-20 00:00:00]",8477464,15.220551,8.573070
TORREY-MITCHELL-8471338.csv,"[[0.11750969, 0.1202602, 0.12651347, 0.1360776...","[0.11750969, 0.1202602, 0.12651347, 0.13607763...","[0.117509685, 0.117509685, 0.117509685, 0.1175...","[2013-10-03 00:00:00, 2018-03-30 00:00:00]","[2018-04-02 00:00:00, 2018-04-02 00:00:00]",8471338,12.709690,9.635795


### 6.2 HMM

In [25]:
# TODO: Make sure predictions are updated
hmm_preds = pd.read_pickle(os.path.join(data_folder, 'predictions_hmm_3years_for_1920.pkl'))
hmm_preds.index.name = 'name'
hmm_preds.head()

,nobs,minmax,mean,variance,skewness,kurtosis
name,,,,,,
RYAN-KESLER-8470616.csv,500,"(19.0486693136375, 57.939195964332185)",35.7392,34.5478,0.135661,0.119872
ETHAN-BEAR-8478451.csv,0,0,0,0,0,0
LOGAN-SHAW-8476400.csv,500,"(3.9503219831557255, 32.00679263430701)",15.5919,19.4693,0.243871,0.142775
NELSON-NOGIER-8478031.csv,0,0,0,0,0,0
MIKE-SMITH-8469608.csv,500,"(-0.1488302137364587, 9.169448730565826)",2.69421,2.3494,0.469817,0.222445


### 6.3 Merge predictions

In [26]:
predictions = lstm_preds.merge(hmm_preds, on='name')
predictions.head()

,multiple-sequences,full-sequence,point-by-point,train,test,Player_Id,LSTM_full,LSTM_point,nobs,minmax,mean,variance,skewness,kurtosis
name,,,,,,,,,,,,,,
MARTIN-BRODEUR-8455710.csv,"[[0.10443657, 0.10036451, 0.09233941, 0.082081...","[0.10443657, 0.10036451, 0.09233941, 0.0820819...","[0.10443657, 0.10443657, 0.10443657, 0.1044365...","[2013-10-04 00:00:00, 2014-12-29 00:00:00]","[2015-01-02 00:00:00, 2015-01-02 00:00:00]",8455710,6.465791,8.563799,0,0,0,0,0,0
MATT-BELESKEY-8473492.csv,"[[0.29253426, 0.29245296, 0.29218978, 0.291729...","[0.29253426, 0.29245296, 0.29218978, 0.2917294...","[0.29253426, 0.29253426, 0.29253426, 0.2925342...","[2013-10-02 00:00:00, 2018-12-10 00:00:00]","[2018-12-14 00:00:00, 2018-12-14 00:00:00]",8473492,23.875305,23.987810,500,"(2.964526686043076, 22.109802402360927)",11.1109,9.37974,0.299361,0.0755341
JANNIK-HANSEN-8471498.csv,"[[0.37088394, 0.3701646, 0.36744553, 0.3628682...","[0.37088394, 0.3701646, 0.36744553, 0.36286822...","[0.37088388, 0.37088388, 0.37088388, 0.3708838...","[2013-10-03 00:00:00, 2018-03-29 00:00:00]","[2018-04-05 00:00:00, 2018-04-05 00:00:00]",8471498,29.364569,30.412481,500,"(16.8455536762676, 51.05206414663027)",31.2676,28.8162,0.175287,0.20759
NIC-PETAN-8477464.csv,"[[0.10454964, 0.10764119, 0.115878664, 0.13068...","[0.10454964, 0.10764119, 0.115878664, 0.130686...","[0.10454964, 0.10454964, 0.10454964, 0.1045496...","[2015-10-08 00:00:00, 2019-03-19 00:00:00]","[2019-03-20 00:00:00, 2019-03-20 00:00:00]",8477464,15.220551,8.573070,500,"(6.917666694027673, 31.063019027147497)",16.9744,19.4736,0.341211,-0.27229
TORREY-MITCHELL-8471338.csv,"[[0.11750969, 0.1202602, 0.12651347, 0.1360776...","[0.11750969, 0.1202602, 0.12651347, 0.13607763...","[0.117509685, 0.117509685, 0.117509685, 0.1175...","[2013-10-03 00:00:00, 2018-03-30 00:00:00]","[2018-04-02 00:00:00, 2018-04-02 00:00:00]",8471338,12.709690,9.635795,500,"(6.005520473715006, 30.0500425803704)",17.1779,16.6951,0.429314,0.030703


### 6.4 Merge to stats

In [27]:
nhl['merge_on'] = nhl.apply(lambda player: re.sub(r'\W+', '', f'{player["First Name"]}{player["Last Name"]}').upper(), axis=1)
predictions['merge_on'] = predictions.apply(lambda player: ''.join(player.name.split('-')[:-1]), axis=1)
predictions.replace({'ALEXBURMISTROV': 'ALEXANDERBURMISTROV',
                    'ALEXBURROWS': 'ALEXANDREBURROWS',
                    'MIKECAMMALLERI': 'MICHAELCAMMALLERI',
                    'ANTHONYDEANGELO': 'TONYDEANGELO',
                    'CHRISDIDOMENICO': 'CHRISTOPHERDIDOMENICO',
#                     'ALEXANDEREDLER': 'ALEXEDLER',
#                     'TOBYENSTROM': 'TOBIASENSTROM',
                    'MICHAELFERLAND': 'MICHEALFERLAND',
                    'VINCENTHINNOSTROZA': 'VINNIEHINNOSTROZA',
#                     'NIKOLAYKULEMIN': 'NIKOLAIKULEMIN',
                    'JONATHANAUDYMARCHESSAULT': 'JONATHANMARCHESSAULT',
#                     'MICHAELMATHESON': 'MIKEMATHESON',
                    'JOSHUAMORRISSEY': 'JOSHMORRISSEY',
                    'ALEXANDERPETROVIC': 'ALEXPETROVIC',
#                     'ALEXANDERRADULOV': 'ALEXRADULOV',
#                     'ALEXANDERWENNBERG': 'ALEXWENNBERG',
#                     'ALEXANDERSTEEN': 'ALEXSTEEN',
#                     'ALEXANDERKERFOOT': 'ALEXKERFOOT',
#                     'CHRISTOPHERTANEV': 'CHRISTANEV',
#                     'JTMILLER': 'JMILLER',
#                     'MITCHELLMARNER': 'MITCHMARNER',
#                     'PKSUBBAN': 'PSUBBAN',
                    "RYANO'REILLY": 'RYANOREILLY',
#                     'TJOSHIE': 'TOSHIE'
                    }, 
                    inplace=True)
nhl = nhl.join(predictions.set_index('merge_on'), on='merge_on', lsuffix='_nhl', rsuffix='_preds')
nhl = nhl.drop(['merge_on', 'multiple-sequences', 'full-sequence', 'point-by-point', 'train', 'test', 'nobs'], axis=1)
nhl['min'], nhl['max'] = nhl.minmax.str

/Users/noiseuce/miniconda3/envs/pool/lib/python3.6/site-packages/pandas/core/internals.py:5152: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = op(a, b)
/Users/noiseuce/miniconda3/envs/pool/lib/python3.6/site-packages/pandas/core/missing.py:51: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask = arr == x


In [28]:
# To fill in excel
nhl['x'] = np.nan
nhl['PTS_proj'] = np.nan
nhl['preds'] = np.nan
nhl['$/PTS_exp'] = np.nan
nhl['Name'] = nhl.apply(lambda player: f'{player["First Name"]} {player["Last Name"]}', axis=1)

## 7. Save Results
---

In [29]:
nhl = nhl[['Last Name', 'First Name', 'Name', 'x', 'Team', 'Age', 'Position', 
           'GP_2018', 'GP__mean', 'GP__variance',
           'PTS_proj', 'Total Points_2018', 'Total Points__mean', 'Total Points__variance', 
            # 'PTS_expected_2018', 'PTS_expected__mean', 'PTS_expected__variance',
           'preds', 'LSTM_point', 'LSTM_full', 'mean', 'min', 'max', 'variance',
           'Total Points/GP_2018', 'Total Points/GP__mean', 'Total Points/GP__variance', 
           '$/PTS_exp',
            # '$/PTS_13', '$/PTS_14', '$/PTS_15', '$/PTS_16', '$/PTS_17', '$/PTS_18', '$/PTS_19',
           'On-Ice SH%_2018', 'On-Ice SH%__mean', 'On-Ice SH%__variance', 
           'TOI/GP_x_2018', #'TOI/GP_x__mean', 'TOI/GP_x__variance', 
           'TOI/GP_PP_2018',
           'Off. Zone Start %_2018', #'Off. Zone Start %__mean', 'Off. Zone Start %__variance', 
           'iSCF/60_2018', #'iSCF/60__mean', 'iSCF/60__variance',
           'PDO_2018', #'PDO__mean', 'PDO__variance',
           'Cap_Hit', 
           'END',
           'Score']]
nhl.columns = ['Last Name', 'First Name', 'Name', 'x', 'TM', 'Age', 'POS', 
               'GP_18', 'GP_mean', 'GP_var',
               'PTS_proj', 'PTS_18', 'PTS_mean', 'PTS_var', 
               'Preds', 'LSTM_point', 'LSTM_full', 'HMM_mean', 'HMM_min', 'HMM_max', 'HMM_var',
               'PTS/GP_18', 'PTS/GP_mean', 'PTS/GP_var', 
               '$/PTS_exp',
               'SH%_18', 'SH%_mean', 'SH%_var', 
               'TOI/GP_18', 
               'PPTOI/GP_18',
               'OZ%_18', 
               'iSCF/60_18', 
               'PDO_18', 
               'Cap_Hit', 
               'End',
               'Score']
nhl.sort_values('Score', ascending=False).head(10)

,Last Name,First Name,Name,x,TM,Age,POS,GP_18,GP_mean,GP_var,PTS_proj,PTS_18,PTS_mean,PTS_var,Preds,LSTM_point,LSTM_full,HMM_mean,HMM_min,HMM_max,HMM_var,PTS/GP_18,PTS/GP_mean,PTS/GP_var,$/PTS_exp,SH%_18,SH%_mean,SH%_var,TOI/GP_18,PPTOI/GP_18,OZ%_18,iSCF/60_18,PDO_18,Cap_Hit,End,Score
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ALEXOVECHKIN,Ovechkin,Alex,Alex Ovechkin,NaN,WSH,33,L,81.0,81.0,1.20,NaN,88.0,78.80,58.9600,NaN,96.362129,91.532623,82.241984,58.932437,130.949870,82.354541,1.086420,0.972610,0.008328,NaN,13.58,12.060,0.695720,20.921193,4.429218,70.08,10.27,1.033,9.53846e+06,21.0,9.179341
SIDNEYCROSBY,Crosby,Sidney,Sidney Crosby,NaN,PIT,31,C,79.0,78.6,5.84,NaN,100.0,89.00,34.4000,NaN,97.612328,102.777458,99.668406,62.245968,130.096255,101.131459,1.265823,1.133086,0.006040,NaN,13.06,11.196,1.024424,20.991772,3.216245,67.08,8.72,1.024,8.7e+06,25.0,8.770515
VLADIMIRTARASENKO,Tarasenko,Vladimir,Vladimir Tarasenko,NaN,STL,27,R,76.0,79.0,4.80,NaN,68.0,71.00,11.2000,NaN,59.210033,65.409004,74.825295,46.053000,101.924244,70.089508,0.894737,0.899046,0.001718,NaN,10.08,10.716,0.568864,18.385307,2.921272,69.80,12.32,1.005,7.5e+06,23.0,8.715251
NIKITAKUCHEROV,Kucherov,Nikita,Nikita Kucherov,NaN,TBL,25,R,82.0,79.0,9.60,NaN,127.0,87.80,552.5600,NaN,95.658485,150.632614,109.899726,74.734270,141.085596,122.611977,1.548780,1.109212,0.077733,NaN,14.38,12.638,1.737856,19.965447,3.712602,69.76,9.35,1.053,9.5e+06,NaN,8.697403
PATRICKKANE,Kane,Patrick,Patrick Kane,NaN,CHI,30,R,81.0,77.6,69.04,NaN,109.0,88.20,294.9600,NaN,89.155556,109.609917,92.596296,69.040971,119.956715,82.235794,1.345679,1.132630,0.025055,NaN,12.87,11.408,2.338296,22.489095,3.712551,73.11,10.24,1.027,1.05e+07,23.0,8.696137
ARTEMIPANARIN,Panarin,Artemi,Artemi Panarin,NaN,NYR,27,L,79.0,80.5,1.25,NaN,87.0,79.75,27.6875,NaN,101.867989,108.854309,85.973341,62.045696,114.116690,73.709948,1.101266,0.991589,0.005894,NaN,11.42,11.335,1.701875,19.856540,2.797257,89.10,6.58,1.003,1.16429e+07,NaN,8.575595
JOHNTAVARES,Tavares,John,John Tavares,NaN,TOR,28,C,82.0,80.2,4.96,NaN,87.0,78.00,76.0000,NaN,86.594780,89.930206,81.297138,57.817798,104.230446,81.541118,1.060976,0.970303,0.006903,NaN,12.51,11.212,0.915056,19.089228,2.594309,60.23,13.99,1.025,1.1e+07,25.0,8.527451
PHILKESSEL,Kessel,Phil,Phil Kessel,NaN,ARI,31,R,82.0,82.0,0.00,NaN,82.0,72.60,150.6400,NaN,84.282234,82.262589,82.727146,58.145005,112.985022,80.234546,1.000000,0.885366,0.022403,NaN,11.09,10.570,1.370120,17.978252,3.246545,84.17,8.83,1.014,6.8e+06,22.0,8.511874
EVGENIMALKIN,Malkin,Evgeni,Evgeni Malkin,NaN,PIT,32,C,68.0,66.8,50.16,NaN,72.0,73.60,178.6400,NaN,88.834389,92.968872,98.084867,76.920736,125.128491,75.829936,1.058824,1.094978,0.009077,NaN,11.56,11.316,0.916024,18.803922,3.427206,81.23,9.20,0.996,9.5e+06,22.0,8.504990


In [30]:
nhl[~(nhl.index == 'NANNAN') & (nhl.POS == 'D')].to_csv(os.path.join(data_folder, 'nhl_def_1920_v2.csv'), sep=';')
nhl[~(nhl.index == 'NANNAN') & (nhl.POS != 'D')].to_csv(os.path.join(data_folder, 'nhl_att_1920_v2.csv'), sep=';')

## 8. Rookies
---

* All datasets were copy/pasted from various sources on the internet.

In [76]:
# Read datasets and standardize index

hn = pd.read_csv('../data/hockeynews-rookies.csv', header=None, sep='\t')
hn = hn.reset_index()
hn['Name'] = hn.apply(lambda player: re.sub(r'\W+', '', f'{player[0]}').upper(), axis=1)
hn = hn.set_index('Name')

fc = pd.read_csv('../data/futureconsideration-rookie.csv', header=None, sep='\t')
fc = fc.reset_index()
fc['Name'] = fc.apply(lambda player: (player[0].split(', ')[-1]+player[0].split(', ')[0]).upper(), axis=1)
fc = fc.set_index('Name')

hp = pd.read_csv('../data/hockeyprospect-rookie.csv', header=None, sep='\t')
hp = hp.reset_index()
hp['Name'] = hp.apply(lambda player: (player[0].split(', ')[-1]+player[0].split(', ')[0]).upper(), axis=1)
hp = hp.set_index('Name')

heop = pd.read_csv('../data/habseyeontheprize-rookie.csv', header=None)
heop = heop.reset_index()
heop['Name'] = heop.apply(lambda player: (''.join([player[1].strip(), player[0].strip()])).upper(), axis=1)
heop = heop.set_index('Name')

hw = pd.read_csv('../data/hockeywriters-rookies.csv', header=None, sep='\t')
hw = hw.reset_index()
hw['Name'] = hw.apply(lambda player: re.sub(r'\W+', '', f'{player[0]}').upper(), axis=1)
hw = hw.set_index('Name')

sn = pd.read_csv('../data/sportingnews-rookie.csv', header=None, sep='\t')
sn = sn.reset_index()
sn['Name'] = sn.apply(lambda player: re.sub(r'\W+', '', f'{player[0]}').upper(), axis=1)
sn = sn.set_index('Name')

dha = pd.read_csv('../data/dobberhockey-att.csv', header=None, sep='\t')
dhd = pd.read_csv('../data/dobberhockey-def.csv', header=None, sep='\t')
dh = pd.concat([dha, dhd])
dh = dh.reset_index()
dh['Name'] = dh.apply(lambda player: re.sub(r'\W+', '', f'{player[0]}').upper(), axis=1)
dh = dh.set_index('Name')

In [77]:
# Join datasets
tmp = dh.join(fc, lsuffix='_dh', rsuffix='_fc', how='outer')
tmp = tmp.join(hp, rsuffix='_hp', how='outer')
tmp = tmp.join(heop, rsuffix='_heop', how='outer')
tmp = tmp.join(hw, rsuffix='_hw', how='outer')
tmp = tmp.join(sn, rsuffix='_sn', how='outer')
rookies = tmp.join(hn, rsuffix='_hn', how='outer')

# Keep only ranks and fill for missing names in datasets
# rookies = rookies.filter(regex=("^index*|^2_*|^0_dh$|^0_sn$|^0$"))

In [79]:
rookies[['index_dh', 'index_fc', 'index', 
         'index_heop', 'index_hw', 'index_sn', 'index_hn']] = rookies[['index_dh', 'index_fc', 'index', 
                                                                       'index_heop', 'index_hw', 'index_sn', 'index_hn']].fillna(rookies.max(axis=0))

# Get score and save results
rookies['score'] = rookies.filter(regex=("^index*")).sum(axis=1)
rookies.to_csv(os.path.join(data_folder, 'rookies.csv'))